In [1]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
import lightgbm as lgb
from math import sqrt
from sklearn.model_selection import KFold
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

plt.rcParams['figure.figsize'] = [16, 10]

In [2]:
df = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')


In [3]:
X = df[['lln_00', 'lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', 'lon',
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']]

y= df["target"]

Xtest= df_test[['lln_00', 'lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', 'lon',
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
rf = RandomForestRegressor(n_estimators=333, min_samples_split=2, max_features= 3,
                           min_samples_leaf=2,max_depth=110, bootstrap= False)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=110,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=333, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [6]:
predictions= rf.predict(X_test)
print("RMSE:", sqrt(mean_squared_error(y_test, predictions)))

RMSE: 3.312147677491365


In [7]:
df_test["new_preds"]=rf.predict(Xtest)

In [8]:
df_test[['ward',"new_preds"]].to_csv('new_predictions.csv', index=False)

In [9]:
df_test[['ward', "new_preds"]]

,ward,new_preds
0,21001001: Ward 1,18.658802
1,21001002: Ward 2,15.188663
2,21001003: Ward 3,22.092333
3,21001004: Ward 4,16.911946
4,21001005: Ward 5,21.514511
...,...,...
1008,19100107: Ward 107,7.534655
1009,19100108: Ward 108,22.302714
1010,19100109: Ward 109,17.419900
1011,19100110: Ward 110,13.123684


## XGBOOST REGRESSOR MODEL

In [103]:
model_xgb = XGBRegressor(n_estimators=1000,learning_rate=0.05, min_child_weight=2,max_depth=4,
                         eta =0.6, reg_lambda = 2)

model_xgb.fit(X_train, y_train)
sqrt(mean_squared_error(y_test, model_xgb.predict(X_test)))

/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:33:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


3.1841984467152384

### Best parameters

{'colsample_bytree': 0.8, 'max_depth': 20, 'n_estimators': 1000, 'reg_alpha': 1.3, 'reg_lambda': 1.3, 'subsample': 0.7}


## Parameter tunning for xgboost

In [13]:
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     StratifiedKFold)

In [14]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=3, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [15]:
model = XGBRegressor()
param_grid = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'subsample': [0.7, 0.8, 0.9]
}

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid, cv=5)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   35.2s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:  2.9min
[Parallel(n_jobs=3)]: Done 359 tasks      | elapsed:  8.5min
[Parallel(n_jobs=3)]: Done 642 tasks      | elapsed: 15.3min
[Parallel(n_jobs=3)]: Done 1007 tasks      | elapsed: 25.1min
[Parallel(n_jobs=3)]: Done 1452 tasks      | elapsed: 37.4min
[Parallel(n_jobs=3)]: Done 1979 tasks      | elapsed: 53.2min
[Parallel(n_jobs=3)]: Done 2430 out of 2430 | elapsed: 67.4min finished
/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:10:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
3.4525076004107658
{'colsample_bytree': 0.8, 'max_depth': 20, 'n_estimators': 1000, 'reg_alpha': 1.3, 'reg_lambda': 1.3, 'subsample': 0.7}


## Combined xgboost and random forest

In [104]:
avg_pre = (model_xgb.predict(X_test) + rf.predict(X_test))/2

sqrt(mean_squared_error(y_test, avg_pre))

3.15474930872424

## CATBOOST

In [110]:
model_cat=CatBoostRegressor(iterations=1000, depth=3, learning_rate=0.01, loss_function='RMSE')
model_cat.fit(X_train, y_train)
sqrt(mean_squared_error(y_test, model_cat.predict(X_test)))

0:	learn: 10.2094862	total: 2.95ms	remaining: 2.94s
1:	learn: 10.1469568	total: 6.16ms	remaining: 3.07s
2:	learn: 10.0763488	total: 9.77ms	remaining: 3.25s
3:	learn: 10.0083796	total: 17.5ms	remaining: 4.35s
4:	learn: 9.9425756	total: 20.1ms	remaining: 4s
5:	learn: 9.8765535	total: 22.2ms	remaining: 3.67s
6:	learn: 9.8100918	total: 24.1ms	remaining: 3.42s
7:	learn: 9.7516231	total: 26ms	remaining: 3.23s
8:	learn: 9.6870955	total: 28.4ms	remaining: 3.13s
9:	learn: 9.6241686	total: 31.7ms	remaining: 3.14s
10:	learn: 9.5616815	total: 33.6ms	remaining: 3.02s
11:	learn: 9.5008734	total: 35.6ms	remaining: 2.93s
12:	learn: 9.4376410	total: 37.4ms	remaining: 2.84s
13:	learn: 9.3792832	total: 40.7ms	remaining: 2.86s
14:	learn: 9.3186990	total: 42.6ms	remaining: 2.8s
15:	learn: 9.2587608	total: 44.5ms	remaining: 2.74s
16:	learn: 9.2018364	total: 46.3ms	remaining: 2.68s
17:	learn: 9.1437448	total: 48.1ms	remaining: 2.62s
18:	learn: 9.0909736	total: 49.7ms	remaining: 2.57s
19:	learn: 9.0373711	tot

205:	learn: 4.6776892	total: 389ms	remaining: 1.5s
206:	learn: 4.6703324	total: 392ms	remaining: 1.5s
207:	learn: 4.6619288	total: 395ms	remaining: 1.5s
208:	learn: 4.6556707	total: 398ms	remaining: 1.51s
209:	learn: 4.6501066	total: 401ms	remaining: 1.51s
210:	learn: 4.6445459	total: 405ms	remaining: 1.51s
211:	learn: 4.6382635	total: 409ms	remaining: 1.52s
212:	learn: 4.6328049	total: 413ms	remaining: 1.52s
213:	learn: 4.6251637	total: 415ms	remaining: 1.52s
214:	learn: 4.6177847	total: 416ms	remaining: 1.52s
215:	learn: 4.6098563	total: 420ms	remaining: 1.52s
216:	learn: 4.6039629	total: 422ms	remaining: 1.52s
217:	learn: 4.5973824	total: 424ms	remaining: 1.52s
218:	learn: 4.5911759	total: 425ms	remaining: 1.52s
219:	learn: 4.5834954	total: 427ms	remaining: 1.51s
220:	learn: 4.5755789	total: 429ms	remaining: 1.51s
221:	learn: 4.5693646	total: 430ms	remaining: 1.51s
222:	learn: 4.5639259	total: 432ms	remaining: 1.5s
223:	learn: 4.5570092	total: 435ms	remaining: 1.5s
224:	learn: 4.550

403:	learn: 3.9349723	total: 773ms	remaining: 1.14s
404:	learn: 3.9333196	total: 776ms	remaining: 1.14s
405:	learn: 3.9313910	total: 777ms	remaining: 1.14s
406:	learn: 3.9294544	total: 779ms	remaining: 1.13s
407:	learn: 3.9277678	total: 782ms	remaining: 1.14s
408:	learn: 3.9265641	total: 785ms	remaining: 1.13s
409:	learn: 3.9249780	total: 788ms	remaining: 1.13s
410:	learn: 3.9227702	total: 792ms	remaining: 1.13s
411:	learn: 3.9216153	total: 796ms	remaining: 1.14s
412:	learn: 3.9201146	total: 801ms	remaining: 1.14s
413:	learn: 3.9189847	total: 803ms	remaining: 1.14s
414:	learn: 3.9175206	total: 805ms	remaining: 1.13s
415:	learn: 3.9156095	total: 807ms	remaining: 1.13s
416:	learn: 3.9134259	total: 808ms	remaining: 1.13s
417:	learn: 3.9116843	total: 810ms	remaining: 1.13s
418:	learn: 3.9102461	total: 814ms	remaining: 1.13s
419:	learn: 3.9082791	total: 815ms	remaining: 1.13s
420:	learn: 3.9069754	total: 817ms	remaining: 1.12s
421:	learn: 3.9051725	total: 819ms	remaining: 1.12s
422:	learn: 

599:	learn: 3.7264749	total: 1.16s	remaining: 771ms
600:	learn: 3.7259510	total: 1.16s	remaining: 769ms
601:	learn: 3.7251021	total: 1.16s	remaining: 768ms
602:	learn: 3.7246024	total: 1.17s	remaining: 768ms
603:	learn: 3.7241267	total: 1.17s	remaining: 766ms
604:	learn: 3.7235437	total: 1.17s	remaining: 765ms
605:	learn: 3.7227515	total: 1.18s	remaining: 764ms
606:	learn: 3.7220713	total: 1.18s	remaining: 764ms
607:	learn: 3.7214975	total: 1.18s	remaining: 762ms
608:	learn: 3.7207709	total: 1.18s	remaining: 760ms
609:	learn: 3.7203296	total: 1.19s	remaining: 758ms
610:	learn: 3.7198013	total: 1.19s	remaining: 756ms
611:	learn: 3.7195023	total: 1.19s	remaining: 753ms
612:	learn: 3.7184797	total: 1.19s	remaining: 751ms
613:	learn: 3.7174122	total: 1.19s	remaining: 751ms
614:	learn: 3.7168102	total: 1.2s	remaining: 749ms
615:	learn: 3.7164139	total: 1.2s	remaining: 747ms
616:	learn: 3.7158754	total: 1.2s	remaining: 745ms
617:	learn: 3.7153957	total: 1.2s	remaining: 743ms
618:	learn: 3.71

797:	learn: 3.6383277	total: 1.54s	remaining: 390ms
798:	learn: 3.6381558	total: 1.54s	remaining: 388ms
799:	learn: 3.6380067	total: 1.54s	remaining: 386ms
800:	learn: 3.6376700	total: 1.54s	remaining: 384ms
801:	learn: 3.6374708	total: 1.55s	remaining: 382ms
802:	learn: 3.6370002	total: 1.55s	remaining: 380ms
803:	learn: 3.6367546	total: 1.55s	remaining: 378ms
804:	learn: 3.6361437	total: 1.55s	remaining: 376ms
805:	learn: 3.6359556	total: 1.55s	remaining: 374ms
806:	learn: 3.6356025	total: 1.56s	remaining: 372ms
807:	learn: 3.6354829	total: 1.56s	remaining: 371ms
808:	learn: 3.6351320	total: 1.56s	remaining: 369ms
809:	learn: 3.6346599	total: 1.56s	remaining: 367ms
810:	learn: 3.6345864	total: 1.57s	remaining: 365ms
811:	learn: 3.6343932	total: 1.57s	remaining: 364ms
812:	learn: 3.6341532	total: 1.57s	remaining: 362ms
813:	learn: 3.6340186	total: 1.58s	remaining: 361ms
814:	learn: 3.6335966	total: 1.58s	remaining: 359ms
815:	learn: 3.6330887	total: 1.58s	remaining: 357ms
816:	learn: 

3.6714183091395345

## Save in csv 

In [59]:
def create_submission(data, fname, cols, models=[]):
    score = 0
    for model in models:
        score += model.predict(data[cols])
    data['target'] = (score / len(models))
    submissions = data[['ward', 'target']]
    submissions.to_csv(fname, index=False)
    return submissions

In [60]:
top_cols_20 = ['lln_00', 'lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', 'lon',
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']

In [61]:
create_submission(df_test, 'randomf_xgboost_tria.csv', cols=top_cols_20, models=[rf, model_xgb])

,ward,target
0,21001001: Ward 1,19.396632
1,21001002: Ward 2,14.328704
2,21001003: Ward 3,19.557664
3,21001004: Ward 4,16.405951
4,21001005: Ward 5,20.789021
...,...,...
1008,19100107: Ward 107,5.970138
1009,19100108: Ward 108,22.025402
1010,19100109: Ward 109,16.399742
1011,19100110: Ward 110,12.660751
